Добавляем в запущенный питон всякие дополнительные модули, которые нам понадобятся

In [2]:
import tensorflow as tf # нейросетка
import keras # модуль, который упрощает работу с нейросеткой
import os
import numpy as np # модуль для особого формата чисел
import pandas as pd # модуль для таблиц, чтобы удобнее было смотреть на цифирки
from keras.models import Sequential
from keras.layers import Dense

Для примера создадим датасет себе сами,
чтобы нам было понятнее что там внутри
как что с чем связано и что мы захотим от нейросетки увидеть,
когда она заработает.

В датасете будет три параметра: один искомый/интересующий, 
который мы ждём от нейросетки на выходе - это y. И два будем подавать на вход (это x1 и x2) - то есть от этих параметров должен зависеть результат (y)

Сделаем датасет так, что y (интересуемый) должен быть почти на единицу меньше, чем два входных параметра.
В результате получим таблицу с тремя столбцами (y, x1, x2) и тысячей строк

In [3]:
x = list()
y = list()

for i in range(1000):  # цикл 1000 раз
    x.append([i+1.2, i+2.1])  # эта переменная хранит в себе два столбца - x1 и x2
    y.append(i)

Создадим удобную таблицу и посмотрим как выглядит её начало:

In [4]:
df = pd.DataFrame(x, columns=['x1', 'x2'])
df['y'] = y
df = df[['y', 'x1', 'x2']]
df.head()

,y,x1,x2
0,0,1.2,2.1
1,1,2.2,3.1
2,2,3.2,4.1
3,3,4.2,5.1
4,4,5.2,6.1


Теперь нужно разбить нашу таблицу на две группы. Одна группа это данные, по которой модель учится (train). Вторая группа (test) это данные, по которой мы будем проверять работу модели - эти данные не должны попасть в корпус обучения. Так мы посмотрим как модель работает на данных, которые не видела.

Пусть, первые 700 строк таблицы это будут данные для обучения, а последние триста это данные для теста. Без разницы.

In [5]:
x_train = np.array(x[:700])
x_test = np.array(x[700:])
y_train = np.array(y[:700])
y_test = np.array(y[700:])

Посмотрим на тестовую выборку:

In [6]:
df = pd.DataFrame(x_test, columns=['x1_test', 'x2_test'])
df['y_test'] = y_test
df = df[['y_test', 'x1_test', 'x2_test']]
df.head()

,y_test,x1_test,x2_test
0,700,701.2,702.1
1,701,702.2,703.1
2,702,703.2,704.1
3,703,704.2,705.1
4,704,705.2,706.1


Сделаем самую простую из возможных нейросеток для данной задачи: два нейрона принимают два параметра (x1 и x2), один нейрон выдаёт наш искомый y

In [16]:
model = Sequential()

# первый слой - два нейрона принимают два параметра
model.add(Dense(2, input_dim=2, kernel_initializer='normal', activation='relu'))

# второй слой - один нейрон
model.add(Dense(1, kernel_initializer='normal'))

model.compile(loss='mean_squared_error', optimizer='adam')

Обучаем (тренируем) модель на тренировочном датасете, который состоит первых 700 строк. Переменная x_train включает в себя два параметра, если что (x1 и x2)

In [17]:
model.fit(x_train, y_train, epochs=100)

Epoch 1/100
 32/700 [>.............................] - ETA: 3s - loss: 124504.8828700/700 [==============================] - 0s 273us/step - loss: 163930.1086
Epoch 2/100
700/700 [==============================] - 0s 36us/step - loss: 163145.9828
Epoch 3/100
700/700 [==============================] - 0s 47us/step - loss: 162945.6529
Epoch 4/100
700/700 [==============================] - 0s 38us/step - loss: 162930.2849
Epoch 5/100
700/700 [==============================] - 0s 52us/step - loss: 162914.9480
Epoch 6/100
700/700 [==============================] - 0s 32us/step - loss: 162899.6125
Epoch 

Чтобы проверить как модель обучилась, нужно подать ей на вход тестовые x1 и x2, которые ей не известны (последние 300 строк таблицы) и посмотреть какой "y" она нам предскажет. Такой ли, какой мы от неё ждём

In [18]:
y_predicted = model.predict(x_test)

Сделаем таблицу:

In [19]:
df = pd.DataFrame(x_test, columns=['x1_test', 'x2_test'])
df['y_predicted'] = y_predicted
df = df[['y_predicted', 'x1_test', 'x2_test']]
df.head()

,y_predicted,x1_test,x2_test
0,2.186424,701.2,702.1
1,2.186424,702.2,703.1
2,2.186424,703.2,704.1
3,2.186424,704.2,705.1
4,2.186424,705.2,706.1


Получилась какая-то хуета. Мы модели подаём на вход 701.2 и 702.1, ожидаем что она выдаст 700, а она выдаёт 2.18. Причём, очевидно, что она такое выдаст на что угодно. В приципе, логично что всё хуёво - слишком мало нейронов и итераций.

Добавим в середину слой из четырёх нейронов и посмотим не улучшится ли предсказательная сила модели. Заодно и увеличим количество итераций (циклов), которые обучается модель

In [41]:
# создаём модель заново, но +4 нейрона в середину

model = Sequential()
model.add(Dense(2, input_dim=2, kernel_initializer='normal', activation='relu'))
model.add(Dense(4, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.compile(loss='mean_squared_error', optimizer='adam')

In [42]:
# опять тренируем новую модель

model.fit(x_train, y_train, epochs=400)

Epoch 1/400
700/700 [==============================] - 0s 560us/step - loss: 162935.0984
Epoch 2/400
700/700 [==============================] - 0s 40us/step - loss: 162721.4978
Epoch 3/400
700/700 [==============================] - 0s 47us/step - loss: 162049.1255
Epoch 4/400
700/700 [==============================] - 0s 53us/step - loss: 160459.4539
Epoch 5/400
700/700 [==============================] - 0s 39us/step - loss: 157381.9571
Epoch 6/400
700/700 [==============================] - 0s 56us/step - loss: 152098.3003
Epoch 7/400
700/700 [==============================] - 0s 46us/step - loss: 144201

In [43]:
# посмотрим что модель предскажет

y_predicted = model.predict(x_test)

In [44]:
df = pd.DataFrame(x_test, columns=['x1_test', 'x2_test'])
df['y_predicted'] = y_predicted
df = df[['y_predicted', 'x1_test', 'x2_test']]
df.head()

,y_predicted,x1_test,x2_test
0,699.524597,701.2,702.1
1,700.522705,702.2,703.1
2,701.520752,703.2,704.1
3,702.518738,704.2,705.1
4,703.516846,705.2,706.1


Ну уже поточнее. По мне так модель всё предсказывает хорошо и корреляция там очевидно предельно близка к 1

In [47]:
# более полно, если интересно

df

,y_predicted,x1_test,x2_test
0,699.524597,701.2,702.1
1,700.522705,702.2,703.1
2,701.520752,703.2,704.1
3,702.518738,704.2,705.1
4,703.516846,705.2,706.1
5,704.514954,706.2,707.1
6,705.512878,707.2,708.1
7,706.510986,708.2,709.1
8,707.508972,709.2,710.1
9,708.507019,710.2,711.1
